In [1]:
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import psycopg2 as psycopg
load_dotenv()

True

In [2]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.environ.get("DB_DESTINATION_HOST"), 
    "port": os.environ.get("DB_DESTINATION_PORT"),
    "dbname": os.environ.get("DB_DESTINATION_NAME"),
    "user": os.environ.get("DB_DESTINATION_USER"),
    "password": os.environ.get("DB_DESTINATION_PASSWORD"),
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "clean_users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

    # создаёт объект курсора для выполнения запросов к базе данных
    # с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
        # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

        # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [27]:
df.head()

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,13,8091-TTVAX,2015-04-01,NaT,One year,No,Credit card (automatic),100.35,5681.10,Fiber optic,...,Yes,No,Yes,Yes,Male,0,Yes,No,Yes,0
1,14,0280-XJGEX,2015-09-01,2019-10-01,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.30,Fiber optic,...,Yes,No,Yes,Yes,Male,0,No,No,Yes,1
2,15,5129-JLPIS,2018-01-01,NaT,Month-to-month,Yes,Electronic check,105.50,2686.05,Fiber optic,...,Yes,Yes,Yes,Yes,Male,0,No,No,No,0
3,17,3655-SNQYZ,2014-05-01,NaT,Two year,No,Credit card (automatic),113.25,7895.15,Fiber optic,...,Yes,Yes,Yes,Yes,Female,0,Yes,Yes,Yes,0
4,19,9959-WOFKT,2014-03-01,NaT,Two year,No,Bank transfer (automatic),106.70,7382.25,Fiber optic,...,Yes,No,Yes,Yes,Male,0,No,Yes,Yes,0


Готовим данные

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from category_encoders import CatBoostEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from catboost import CatBoostClassifier

In [81]:
features = df.drop(['target','id','customer_id','end_date','begin_date'],axis=1)

cat_features = features.select_dtypes(include='object')
potential_binary_features = cat_features.nunique() == 2
binary_cat_features = cat_features[potential_binary_features[potential_binary_features].index]
other_cat_features = cat_features[potential_binary_features[~potential_binary_features].index]
num_features = features.select_dtypes(['float'])

binary_cols = binary_cat_features.columns.tolist()
non_binary_cat_cols = other_cat_features.columns.tolist()
num_cols = num_features.columns.tolist()

In [82]:
preprocessor = ColumnTransformer([
            ('binary', OneHotEncoder(drop='if_binary', sparse_output=False), binary_cols),
            ('cat', OneHotEncoder(drop='if_binary'), non_binary_cat_cols),
            ('num', StandardScaler(), num_cols)
        ],
        remainder='drop',
        verbose_feature_names_out=False
    )
preprocessor.fit(features,df['target'])

ColumnTransformer(transformers=[('binary',
                                 OneHotEncoder(drop='if_binary',
                                               sparse_output=False),
                                 ['paperless_billing', 'internet_service',
                                  'online_security', 'online_backup',
                                  'device_protection', 'tech_support',
                                  'streaming_tv', 'streaming_movies', 'gender',
                                  'partner', 'dependents', 'multiple_lines']),
                                ('cat', OneHotEncoder(drop='if_binary'),
                                 ['type', 'payment_method']),
                                ('num', StandardScaler(),
                                 ['monthly_charges', 'total_charges'])],
                  verbose_feature_names_out=False)

Разбиваем выборки на тренировочную и тестовую

In [83]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(
    features,
    df['target'],
    stratify=df['target'])

Тренируем модель

In [85]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(auto_class_weights='Balanced') 
X_tr_transformed = preprocessor.transform(X_tr)
model.fit(X_tr_transformed, y_tr)

X_val_transformed = preprocessor.transform(X_val)
prediction = model.predict(X_val_transformed)
probas = model.predict_proba(X_val_transformed)

Learning rate set to 0.020969
0:	learn: 0.6867324	total: 1.65ms	remaining: 1.65s
1:	learn: 0.6797008	total: 3.73ms	remaining: 1.86s
2:	learn: 0.6717883	total: 5.86ms	remaining: 1.95s
3:	learn: 0.6638465	total: 8.06ms	remaining: 2.01s
4:	learn: 0.6571475	total: 10.3ms	remaining: 2.05s
5:	learn: 0.6497185	total: 12.5ms	remaining: 2.07s
6:	learn: 0.6431867	total: 14.7ms	remaining: 2.09s
7:	learn: 0.6371379	total: 17.1ms	remaining: 2.12s
8:	learn: 0.6303931	total: 19.4ms	remaining: 2.14s
9:	learn: 0.6243840	total: 21.7ms	remaining: 2.15s
10:	learn: 0.6186223	total: 24ms	remaining: 2.16s
11:	learn: 0.6124860	total: 26.4ms	remaining: 2.17s
12:	learn: 0.6065063	total: 28.7ms	remaining: 2.18s
13:	learn: 0.6014167	total: 31ms	remaining: 2.18s
14:	learn: 0.5966919	total: 33.3ms	remaining: 2.19s
15:	learn: 0.5918642	total: 35.6ms	remaining: 2.19s
16:	learn: 0.5867156	total: 38ms	remaining: 2.19s
17:	learn: 0.5840133	total: 39.9ms	remaining: 2.18s
18:	learn: 0.5793402	total: 45.6ms	remaining: 2.35

Создаём метрики

In [86]:
print(probas[:,1])

[0.51910722 0.98184281 0.86650628 ... 0.74700945 0.01334755 0.60483328]


In [87]:
from sklearn.metrics import f1_score,roc_auc_score,recall_score,confusion_matrix,log_loss,precision_score

# заведите словарь со всеми метриками
metrics = {}

# посчитайте метрики из модуля sklearn.metrics
# err_1 — ошибка первого рода
# err_2 — ошибка второго рода
_, err1, _, err2 = confusion_matrix(y_val,prediction,normalize='all').ravel()
auc = roc_auc_score(y_val,probas[:,1])
precision = precision_score(y_val,prediction)
recall = recall_score(y_val,prediction)
f1 = f1_score(y_val,prediction)
logloss = log_loss(y_val,prediction)

# запишите значения метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

print(metrics)

{'err1': 0.16638273708120385, 'err2': 0.19875070982396364, 'auc': 0.8412728488262414, 'precision': 0.5443234836702955, 'recall': 0.7494646680942184, 'f1': 0.6306306306306305, 'logloss': 8.391764843576395}


Делаем словарь по исходным данным

In [51]:
counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}

for col in counts_columns:
		# посчитайте уникальные значения для колонок, где немного уникальных значений (переменная counts_columns)
    column_stat = df[col].value_counts()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}

		# обновите словарь stats
    stats.update(column_stat)


stats["data_length"] = df.shape[0]
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max()
stats["monthly_charges_mean"] = df["monthly_charges"].mean() # посчитайте среднее значение в колонке
stats["monthly_charges_median"] = df["monthly_charges"].median() # посчитайте медианное значение в колонке
stats["total_charges_min"] = df["total_charges"].min() # посчитайте минимальное значение в колонке
stats["total_charges_max"] = df["total_charges"].max() # посчитайте максимальное значение в колонке
stats["total_charges_mean"] = df["total_charges"].mean() # посчитайте среднее значение в колонке
stats["total_charges_median"] = df["total_charges"].median() # посчитайте медианное значение в колонке
stats["unique_customers_number"] = df["customer_id"].nunique() # посчитайте кол-во уникальных id
stats["end_date_nan"] = df["end_date"].isnull().sum() # посчитайте кол-во пустых строк в колонке

In [12]:
print(stats)

{'type_Month-to-month': 3875, 'type_Two year': 1695, 'type_One year': 1473, 'paperless_billing_Yes': 4171, 'paperless_billing_No': 2872, 'internet_service_Fiber optic': 4622, 'internet_service_DSL': 2421, 'online_security_No': 5024, 'online_security_Yes': 2019, 'online_backup_No': 4614, 'online_backup_Yes': 2429, 'device_protection_No': 4621, 'device_protection_Yes': 2422, 'tech_support_No': 4999, 'tech_support_Yes': 2044, 'streaming_tv_No': 4336, 'streaming_tv_Yes': 2707, 'streaming_movies_No': 4311, 'streaming_movies_Yes': 2732, 'gender_Male': 3555, 'gender_Female': 3488, 'senior_citizen_0': 5901, 'senior_citizen_1': 1142, 'partner_No': 3641, 'partner_Yes': 3402, 'dependents_No': 4933, 'dependents_Yes': 2110, 'multiple_lines_No': 4072, 'multiple_lines_Yes': 2971, 'target_0': 5174, 'target_1': 1869, 'data_length': 7043, 'monthly_charges_min': 18.25, 'monthly_charges_max': 118.75, 'monthly_charges_mean': 64.76169246059918, 'monthly_charges_median': 70.35, 'total_charges_min': 18.8, 'to

In [88]:
pipline = Pipeline([
            ('preprocessor', preprocessor),
            ('model', model)
        ]
    )
pipline.fit(X_tr, y_tr)

Learning rate set to 0.020969
0:	learn: 0.6867324	total: 1.64ms	remaining: 1.64s
1:	learn: 0.6797008	total: 3.53ms	remaining: 1.76s
2:	learn: 0.6717883	total: 5.6ms	remaining: 1.86s
3:	learn: 0.6638465	total: 9.89ms	remaining: 2.46s
4:	learn: 0.6571475	total: 12.3ms	remaining: 2.44s
5:	learn: 0.6497185	total: 14.6ms	remaining: 2.42s
6:	learn: 0.6431867	total: 16.9ms	remaining: 2.4s
7:	learn: 0.6371379	total: 19.3ms	remaining: 2.39s
8:	learn: 0.6303931	total: 24.7ms	remaining: 2.72s
9:	learn: 0.6243840	total: 27ms	remaining: 2.67s
10:	learn: 0.6186223	total: 29.5ms	remaining: 2.66s
11:	learn: 0.6124860	total: 32ms	remaining: 2.63s
12:	learn: 0.6065063	total: 34.4ms	remaining: 2.61s
13:	learn: 0.6014167	total: 36.7ms	remaining: 2.59s
14:	learn: 0.5966919	total: 39.2ms	remaining: 2.57s
15:	learn: 0.5918642	total: 41.5ms	remaining: 2.55s
16:	learn: 0.5867156	total: 43.8ms	remaining: 2.53s
17:	learn: 0.5840133	total: 45.3ms	remaining: 2.47s
18:	learn: 0.5793402	total: 47.6ms	remaining: 2.46

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('binary',
                                                  OneHotEncoder(drop='if_binary',
                                                                sparse_output=False),
                                                  ['paperless_billing',
                                                   'internet_service',
                                                   'online_security',
                                                   'online_backup',
                                                   'device_protection',
                                                   'tech_support',
                                                   'streaming_tv',
                                                   'streaming_movies', 'gender',
                                                   'partner', 'dependents',
                                                   'multiple_lines']),
                                                 ('cat',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['type', 'payment_method']),
                                                 ('num', StandardScaler(),
                                                  ['monthly_charges',
                                                   'total_charges'])],
                                   verbose_feature_names_out=False)),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x7f31b016d300>)])

In [53]:
pipline_prediction = pipline.predict(X_val)
pipline_probas     = pipline.predict_proba(X_val)

Готовим подключение к MLFlow

In [13]:
os.environ['MLFLOW_S3_ENDPOINT_URL']='https://storage.yandexcloud.net'
os.environ['AWS_BUCKET_NAME']=os.environ.get("S3_BUCKET_NAME")

In [14]:
import mlflow
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [89]:
EXPERIMENT_NAME = "churn_ujhmaster"
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
#experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
print(experiment_id)

30


Записываем файлы артефактов

In [54]:
with open("columns.txt", "w", encoding="utf-8") as fio:
    fio.write(','.join(columns[1:]))

df.to_csv("users_churn.csv", index=False)

import joblib
os.makedirs('models', exist_ok=True)
joblib.dump(pipline,'models/fitted_model.pkl')

['models/fitted_model.pkl']

Логируем артефакты в MLFlow

In [55]:
RUN_NAME = "data_check"
DIR_PATH = 'dataframe'

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id
    
    # логируем метрики эксперимента
    # предполагается, что переменная stats содержит словарь с метриками,
    # где ключи — это названия метрик, а значения — числовые значения метрик
    mlflow.log_metrics(stats)
    mlflow.log_metrics(metrics)
    
    # логируем файлы как артефакты эксперимента — 'columns.txt' и 'users_churn.csv'
    mlflow.log_artifact("columns.txt",artifact_path=DIR_PATH)
    mlflow.log_artifact("users_churn.csv",artifact_path=DIR_PATH)
    mlflow.log_artifact("models/fitted_model.pkl",artifact_path=DIR_PATH)


experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id)


# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того, 
# что эксперимент был завершён успешно
assert 'FINISHED' == run.info.status

# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы,
# чтобы очистить рабочую среду после логирования артефактов
os.remove("columns.txt")
os.remove("users_churn.csv")
os.remove("models/fitted_model.pkl")

Логирование модели

In [90]:
pip_requirements = "./requirements.txt"
metadata = {'model_type': 'monthly'}
input_example = X_val[:10]
signature = mlflow.models.infer_signature(X_val, prediction)

REGISTRY_MODEL_NAME = "churn_model_ujhmaster"

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_metrics(stats)
    model_info = mlflow.catboost.log_model( 
            cb_model=model, 
			pip_requirements=pip_requirements,
            signature=signature, 
            metadata=metadata,
            input_example=input_example, 
            artifact_path='models', 
            registered_model_name=REGISTRY_MODEL_NAME,
            await_registration_for=60
		)

/home/mle-user/mle_projects/mle-mlflow/.venv_mlflow/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Registered model 'churn_model_ujhmaster' already exists. Creating a new version of this model...
2024/09/03 14:43:44 INFO mlflow.tracking._model

In [58]:
loaded_model = mlflow.catboost.load_model(model_uri=model_info.model_uri)
model_predictions = loaded_model.predict(X_val_transformed)

assert model_predictions.dtype == int

print(model_predictions[:10])

[0 1 0 0 1 1 1 1 0 0]


обернём для сохранения piplene

In [91]:
class CustomMlflowModel(mlflow.pyfunc.PythonModel):

    def __init__(self, model):
        super().__init__()
        self._model = model 
    
    def predict(self, context, model_input):
        probas = self._model.predict(model_input)
        
        return probas

    
custom_model = CustomMlflowModel(pipline)

In [92]:
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_metrics(stats)
    mlflow.log_metrics(metrics)
    model_info = mlflow.pyfunc.log_model( 
            python_model=custom_model, 
			pip_requirements=pip_requirements,
            signature=signature, 
            metadata=metadata,
            input_example=input_example, 
            artifact_path='models', 
            registered_model_name=REGISTRY_MODEL_NAME,
            await_registration_for=60
		)

Registered model 'churn_model_ujhmaster' already exists. Creating a new version of this model...
2024/09/03 14:44:09 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: churn_model_ujhmaster, version 9
Created version '9' of model 'churn_model_ujhmaster'.


In [93]:
loaded_model = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)
model_predictions = loaded_model.predict(X_val)

assert model_predictions.dtype == int

print(model_predictions[:10])

[1 1 1 1 0 1 1 1 1 0]


In [94]:
print(y_val[:10])

2173    1
6080    1
3020    0
2519    0
4888    0
3688    0
2248    1
1038    0
3421    1
4909    0
Name: target, dtype: int64


In [95]:
import boto3
def get_session_student():
    session = boto3.session.Session()

    return session.client(
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net',
        aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY")
    )

s3 = get_session_student()
bucket_name = os.environ.get('S3_BUCKET_NAME')
if s3.list_objects(Bucket=bucket_name)['Contents']:
    for key in s3.list_objects(Bucket=bucket_name)['Contents']:
        print(key['Key'])

24/5f27fe91a8884a88b1a7bbb012399f35/artifacts/test/columns.txt
27/99d4253759d64790aa9bb92c68458454/artifacts/dataframe/columns.txt
27/99d4253759d64790aa9bb92c68458454/artifacts/dataframe/users_churn.csv
28/7ecb555f01fa41a4b91b45faeaea2a34/artifacts/dataframe/columns.txt
28/7ecb555f01fa41a4b91b45faeaea2a34/artifacts/dataframe/users_churn.csv
29/14244d341f734cf5aae73cd8bba8f0bb/artifacts/dataframe/columns.txt
29/14244d341f734cf5aae73cd8bba8f0bb/artifacts/dataframe/users_churn.csv
30/0543bae5f7224823bf25b8b10ebce410/artifacts/models/MLmodel
30/0543bae5f7224823bf25b8b10ebce410/artifacts/models/conda.yaml
30/0543bae5f7224823bf25b8b10ebce410/artifacts/models/input_example.json
30/0543bae5f7224823bf25b8b10ebce410/artifacts/models/python_env.yaml
30/0543bae5f7224823bf25b8b10ebce410/artifacts/models/python_model.pkl
30/0543bae5f7224823bf25b8b10ebce410/artifacts/models/requirements.txt
30/0ddf7507cb354a42ad52c3a5923fbaac/artifacts/models/MLmodel
30/0ddf7507cb354a42ad52c3a5923fbaac/artifacts/mode